# 1 Reading and writing Data in Text Format

In [1]:
import pandas as pd
import numpy as np

Let’s start with a small comma-separated values (CSV) text file:

In [2]:
df = pd.read_csv("examples/ex1.csv")

df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A file will not always have a header row. Consider this file:

In [3]:
pd.read_csv("examples/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_csv("examples/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the `message` column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named `"message"` using the `index_col` argument:

In [5]:
names = ["a", "b", "c", "d", "message"]

pd.read_csv("examples/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


If you want to form a hierarchical index from multiple columns, pass a list of column numbers or names:

```Python
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16
```

In [6]:
parsed = pd.read_csv("examples/csv_mindex.csv", index_col=["key1", "key2"])

parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. Consider a text file that looks like this:

```Python
In [21]: !cat examples/ex3.txt
A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491
```

In these cases, you can pass a regular expression as a delimiter for `pandas.read_csv`. This can be expressed by the regular expression `\s+`, so we have then:

In [7]:
result = pd.read_csv("examples/ex3.txt", sep="\s+")

result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


> Because there was one fewer column name than the number of data rows, pandas.read_csv infers that the first column should be the DataFrame’s index in this special case.

The file parsing functions have many additional arguments to help you handle the wide variety of exception file formats that occur. For example, you can skip the first, third, and fourth rows of a file with `skiprows`:

```Python
In [24]: !cat examples/ex4.csv
# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo
```

In [8]:
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file reading process. Missing data is usually either not present (empty string) or marked by some sentinel (placeholder) value. By default, pandas uses a set of commonly occurring sentinels, such as `NA` and `NULL`:

```Python
In [26]: !cat examples/ex5.csv
something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo
```

In [9]:
result = pd.read_csv("examples/ex5.csv")

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Recall that pandas outputs missing values as `NaN`, so we have two null or missing values in `result`:

In [10]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The `na_values` option accepts a sequence of strings to add to the default list of strings recognized as missing:

`na_values` 选项接受一个字符串序列，用来加入到默认识别为缺失值（NA）的字符串列表中。

换句话说：

Pandas 默认已经把一些字符串当作缺失值，例如：`""、"NA"、"NaN"、"NULL"` 等等。

但是有些数据集会用别的方式表示缺失值，例如 `"?"、"-"、"missing"`。

你可以用 `na_values=` 来告诉 Pandas：
“除了默认识别的缺失值之外，这些字符串也算缺失值。”

In [11]:
result = pd.read_csv("examples/ex5.csv", na_values=["NULL"])

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Pandas 的 read_csv() 函数：

* 默认内置了一大堆字符串，会自动识别为缺失值（NA / NaN），例如：
```Python
"", "NA", "N/A", "NaN", "NULL", "-", "#N/A", "n/a" ...
```
* 但是，如果你 不想要这些默认的缺失值规则，你可以使用：`keep_default_na=False` 来 关闭默认的缺失值识别列表

In [12]:
result2 = pd.read_csv("examples/ex5.csv", keep_default_na=False)

result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [13]:
result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [14]:
result3 = pd.read_csv("examples/ex5.csv", keep_default_na=False, na_values=["NA"])

result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [15]:
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


Different NA sentinels can be specified for each column in a dictionary:

In [16]:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}

pd.read_csv("examples/ex5.csv", na_values=sentinels, keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


## 1.1 Reading Text Files in Pieces

When processing very large files or figuring out the right set of arguments to correctly process a large file, you may want to read only a small piece of a file or iterate through smaller chunks of the file.

Before we look at a large file, we make the pandas display settings more compact:

In [17]:
pd.options.display.max_rows = 10

Now we have:

In [18]:
result = pd.read_csv("examples/ex6.csv")

result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


If you want to read only a small number of rows (avoiding reading the entire file), specify that with `nrows`:

In [19]:
pd.read_csv("examples/ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


1. chunksize 的作用

pd.read_csv() 通常会一次性把整个 CSV 文件读入内存。

但如果文件很大（比如几 GB），一次性载入会：

占用太多内存

效率很低

甚至可能直接内存溢出

为了解决这个问题，你可以使用：`chunksize=1000`
意思是：
> 一次只读取 1000 行，分块加载。

2. TextFileReader 对象是什么？
```Python
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
```
这里返回的不是 DataFrame，而是：
```Python
pandas.io.parsers.readers.TextFileReader
```
> `TextFileReader` 是一个可迭代对象（iterator），每次迭代返回一个 1000 行的 DataFrame 块（chunk）。

你可以像遍历列表一样遍历 chunker：
```Python
for piece in chunker:
    ...
```
> 每次循环 piece 是一个大小为 1000 行的 DataFrame。

In [20]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)

type(chunker)

pandas.io.parsers.readers.TextFileReader

3. 示例：分块统计 key 列的频率

第一步：创建一个空的 Series（用于累加）

In [21]:
tot = pd.Series([], dtype='int64')

第二步：对每一块统计 key 列的 value_counts()

In [22]:
# 每次循环获得一个小 DataFrame
for piece in chunker:
    # 累加到 tot, 因为每块都有不同 key，这一步实现按 key 聚合统计。
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

# 最终排序
tot = tot.sort_values(ascending=False)

# 取前十个
tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

> 代表 key 列中，字母 E 出现最多。

## 1.2 Writing Data to Text Format

Data can also be exported to a delimited format. Let’s consider one of the CSV files read before:

In [23]:
data = pd.read_csv("examples/ex5.csv")

data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Using DataFrame’s `to_csv` method, we can write the data out to a comma-separated file:

In [24]:
data.to_csv("examples/out.csv")

Other delimiters can be used, of course (writing to `sys.stdout` so it prints the text result to the console rather than a file):

In [25]:
import sys

data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Missing values appear as empty strings in the output. You might want to denote them by some other sentinel value:

In [26]:
data.to_csv(sys.stdout, na_rep="NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


With no other options specified, both the row and column labels are written. Both of these can be disabled:

In [27]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


You can also write only a subset of the columns, and in an order of your choosing:

In [29]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


## 1.3 JSON Data

In [30]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

There are several Python libraries for reading and writing JSON data. I’ll use `json` here, as it is built into the Python standard library. To convert a JSON string to Python form, use `json.loads`:

In [31]:
import json

result = json.loads(obj)

result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

`json.dumps`, on the other hand, converts a Python object back to JSON:

In [33]:
asjson = json.dumps(result)

asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

How you convert a JSON object or list of objects to a DataFrame or some other data structure for analysis will be up to you. Conveniently, you can pass a list of dictionaries (which were previously JSON objects) to the DataFrame constructor and select a subset of the data fields:

In [34]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])

siblings

,name,age
0,Scott,34
1,Katie,42


The `pandas.read_json` can automatically convert JSON datasets in specific arrangements into a Series or DataFrame. For example

```Python
In [75]: !cat examples/example.json
[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]
```

In [35]:
data = pd.read_json("examples/example.json")

data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


If you need to export data from pandas to JSON, one way is to use the `to_json` methods on Series and DataFrame:

In [36]:
data.to_json(sys.stdout)

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

In [37]:
data.to_json(sys.stdout, orient="records")

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

# 2 Binary Data Formats

One simple way to store (or serialize) data in binary format is using Python’s built-in pickle module. pandas objects all have a `to_pickle` method that writes the data to disk in pickle format:

In [38]:
frame = pd.read_csv("examples/ex1.csv")

frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [39]:
frame.to_pickle("examples/frame_pickle")

Pickle files are in general readable only in Python. You can read any "pickled" object stored in a file by using the built-in pickle directly, or even more conveniently using `pandas.read_pickle`:

pickle 是 Python 的二进制序列化格式，可以把任意 Python 对象保存到文件，再以原样读回来。但只能在 Python 中使用，不适合跨平台共享，也不安全。

pickle 的特点  
✔ 优点

可以保存任何 Python 对象
而 CSV、JSON 等格式只能保存文本和简单结构。

保存结构完整
如 DataFrame 的 index、dtype、列名等都会完整保留。

读写速度很快
因为是二进制，不需要解析文本。

✔ 缺点

只能在 Python 中读取（非常重要）

R、Java、C++、Scala 都看不懂 pickle

不能跨语言

不同 Python 版本可能不兼容
例如 Python 3 可能无法加载 Python 2 的 pickle。

不是安全格式
加载陌生 pickle 文件可能执行恶意代码。

In [40]:
pd.read_pickle("examples/frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## 2.1 Reading Microsoft Excel Files

In [41]:
xlsx = pd.ExcelFile("examples/ex1.xlsx")

This object can show you the list of available sheet names in the file:

In [42]:
xlsx.sheet_names

['Sheet1']

Data stored in a sheet can then be read into DataFrame with `parse`

In [43]:
xlsx.parse(sheet_name="Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


This Excel table has an index column, so we can indicate that with the `index_col` argument:

In [44]:
# 把 Excel 中的第 0 列（第一列）当成 DataFrame 的索引（index），而不是普通数据列。
xlsx.parse(sheet_name="Sheet1", index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


f you are reading multiple sheets in a file, then it is faster to create the pandas.ExcelFile, but you can also simply pass the filename to `pandas.read_excel`:

In [45]:
frame = pd.read_excel("examples/ex1.xlsx", sheet_name="Sheet1")

frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


To write pandas data to Excel format, you must first create an `ExcelWriter`, then write data to it using the pandas object's `to_excel` method:

In [47]:
writer = pd.ExcelWriter("examples/ex2.xlsx")

frame.to_excel(writer, sheet_name="Sheet1")

writer.close()

You can also pass a file path to `to_excel` and avoid the ExcelWriter:

In [48]:
frame.to_excel("examples/ex2.xlsx")